In [1]:
require(tidyverse)

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [17]:
df_nmr = read_csv('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/11_20_2019_even_odds_interaction_prior_1/nmr_metabolites_scaled_matrix.csv')
df_nmr_raw = read_csv('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/data/20200227_teddy_jenanaro/raw_nmr.csv')

overall_cov = cov(df_nmr[,c(1:27)])
overall_mean = colMeans(df_nmr[,c(1:27)]) 
died = df_nmr$died_90_day
sigma_0 = cov(df_nmr[died == 0,c(1:27)])
mu_0 = colMeans(df_nmr[died == 0,c(1:27)]) 
sigma_1 = cov(df_nmr[died == 1,c(1:27)])
mu_1 = colMeans(df_nmr[died == 1,c(1:27)]) 

metabolites = colnames(df_nmr)[1:27]
metabolites

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_double(),
  t0nmronly = col_character()
)

See spec(...) for full column specifications.



[1] "hydroxybutyrate"   "oxoisocaproate"    "X3.hydoxybutyrate"
 [4] "alanine"           "betaine"           "carnitine"        
 [7] "choline"           "citrate"           "creatine"         
[10] "creatinine"        "glucose"           "glutamine"        
[13] "glycine"           "histidine"         "isoleucine"       
[16] "lactate"           "leucine"           "lysine"           
[19] "methionine"        "oacetylcarnitine"  "ornithine"        
[22] "phenylalanine"     "proline"           "propyleneglycol"  
[25] "pyruvate"          "tyrosine"          "valine"

In [19]:
missing_rate = df_nmr[,metabolites] %>% summarise_all(function(x) { mean(x == min(x))  })
missing_rate_numeric = missing_rate %>% t() %>% as.numeric

In [20]:
missing_rate_numeric

[1] 0.004385965 0.149122807 0.008771930 0.004385965 0.013157895 0.149122807
 [7] 0.157894737 0.065789474 0.004385965 0.004385965 0.004385965 0.004385965
[13] 0.004385965 0.052631579 0.096491228 0.004385965 0.004385965 0.013157895
[19] 0.087719298 0.026315789 0.184210526 0.004385965 0.118421053 0.004385965
[25] 0.004385965 0.030701754 0.004385965

In [21]:
df_out = data.frame(
    METABOLITE=metabolites,
    OVERALL_MEAN=0,
    OVERALL_SD=1,
    MISSING_RATE=missing_rate_numeric,
    SURVIVOR_MEAN=mu_0,
    SURVIVOR_SD=diag(sigma_0),
    NON_SURVIVOR_MEAN=mu_1,
    NON_SURVIVOR_SD=diag(sigma_1),
    DIFFERENCE=mu_1-mu_0
    )
row.names(df_out) = NULL

In [23]:
df_sorted = df_out %>% arrange(-DIFFERENCE)
df_sorted

METABOLITE,OVERALL_MEAN,OVERALL_SD,MISSING_RATE,SURVIVOR_MEAN,SURVIVOR_SD,NON_SURVIVOR_MEAN,NON_SURVIVOR_SD,DIFFERENCE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
oacetylcarnitine,0,1,0.026315789,-0.34882797,0.6775546,0.30846771,1.0887160,0.65729567
betaine,0,1,0.013157895,-0.31443745,0.6800239,0.27805626,1.1248594,0.59249372
tyrosine,0,1,0.030701754,-0.31203422,0.8162382,0.27593109,1.0070663,0.58796531
propyleneglycol,0,1,0.004385965,-0.30208461,0.6693982,0.26713267,1.1470412,0.56921728
glycine,0,1,0.004385965,-0.27406785,0.7929538,0.24235752,1.0650216,0.51642537
lysine,0,1,0.013157895,-0.27018841,0.7751513,0.23892694,1.0842946,0.50911535
glutamine,0,1,0.004385965,-0.26151691,0.7392565,0.23125876,1.1237484,0.49277567
methionine,0,1,0.087719298,-0.24829798,1.0106979,0.21956928,0.8952982,0.46786726
lactate,0,1,0.004385965,-0.24659775,0.5933663,0.21806578,1.2653548,0.46466352


In [24]:
write_csv(df_sorted,path = '/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis/20200729_sim_params/simulation_params.csv')